In [2]:
import torch.optim as optim
from torch import nn
import torch
import torchvision
import torchvision.transforms as transforms
import math
from PIL import Image
from torchvision.datasets import CIFAR100
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle
from torchsummary import summary

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#All the parameters
epoch = 50
lr = 0.1
weight_decay = 0.0001
momentum = 0.9
gamma = 0.99
resnet_type = 5

In [5]:
class ResidualBlock(nn.Module):
  def __init__(self, channel_num, stride, downsample_layer = None):
    super(ResidualBlock, self).__init__()

    self.downsample = downsample_layer
    
    self.conv1 = nn.Sequential(
        nn.Conv2d(int(channel_num/stride), channel_num, 3, stride, padding=1),
        nn.BatchNorm2d(channel_num),
        nn.ReLU(),
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(channel_num, channel_num, 3, int(stride/stride), padding=1),
        nn.BatchNorm2d(channel_num),
    )
    self.relu = nn.ReLU() #Definiramo izvan bloka zbog skip connectiona

  def forward(self, x):
    skip_connection = x
    x = self.conv1(x)
    x = self.conv2(x)
    if(skip_connection.shape != x.shape):
      skip_connection = self.downsample(skip_connection)
    x = skip_connection + x
    x = self.relu(x)
    return x

In [13]:
class ResidualNetwork(nn.Module):
  def __init__(self, channel_num=16):
    super(ResidualNetwork, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(3, channel_num, 3, padding=1),
      nn.BatchNorm2d(channel_num),
      nn.ReLU(),
    )

    res_blocks = []

    for i in range(3 * resnet_type):
      if(i != 0 and i % resnet_type == 0):
        channel_num = channel_num*2
        res_blocks.append(ResidualBlock(channel_num, 2, nn.Sequential(
            nn.Conv2d(int(channel_num/2), channel_num, 1, 2),
            nn.BatchNorm2d(channel_num),
        )))
      else:
        res_blocks.append(ResidualBlock(channel_num, 1))

    self.residualblocks = nn.Sequential(*res_blocks)

    self.pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(64, 100, bias=True)

  def forward(self, x):
    x = self.conv1(x)
    x = self.residualblocks(x)
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    out = self.fc(x)

    return out

In [7]:
trainTransform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))])

testTransform = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))])


trainSet = torchvision.datasets.CIFAR100(root='./drive/MyDrive/Zavrsni/data', train=True,
                    download=False, transform=trainTransform)
testSet = torchvision.datasets.CIFAR100(root='./drive/MyDrive/Zavrsni/data', train=False,
                    download=False, transform=testTransform)

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=128,
                                          shuffle=True, num_workers=0)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=128,
                                         shuffle=True, num_workers=0)

classes=pickle.load(open('./drive/MyDrive/Zavrsni/data/cifar-100-python/meta', 'rb'))
classes=classes['fine_label_names']


In [8]:
def evaluate(net, type, save_path):
    """
    Performs the evaluation of the current performance of a
    given convolutional network. It can perform the evaluation on 
    both training and testing sets. Standard evaluation metrics are
    calcualted such as, accuracy and confusion matrix.
    Parameters
    ----------
    net: ConvolutionalModel
        ConvNet whose performance needs to be evaluated.
    type: bool
        True if eval is made on testing set, false otherwise
    Return
    ------
    loss
        Current loss on the chosen set
    accuracy
        Current acc on the chosen set
    """
    device = torch.device('cuda')
    f = open(save_path, "a+")
    net.eval()
    total = 0
    correct = 0
    confMatrix = np.zeros((100, 100), int)
    lossFunc = nn.CrossEntropyLoss()
    accLoss = 0
    if type:
        with torch.no_grad():
            for data in testLoader:
                images, labels = data
                images = images.to(device=device)
                labels = labels.to(device=device)

                output = net.forward(images)
                loss = lossFunc(output, labels)
                _, predictions = torch.max(output.data, 1)
                total += labels.size(0)
                accLoss += loss.item()
                correct += (predictions == labels).sum().item()
                for j in range(labels.size(0)):
                    confMatrix[predictions[j], labels[j]] += 1
    else:
        with torch.no_grad():
            for data in trainLoader:
                images, labels = data
                images = images.to(device=device)
                labels = labels.to(device=device)

                output = net.forward(images)
                loss = lossFunc(output, labels)
                _, predictions = torch.max(output.data, 1)
                total += labels.size(0)
                accLoss += loss.item()
                correct += (predictions == labels).sum().item()
                for j in range(labels.size(0)):
                    confMatrix[predictions[j], labels[j]] += 1

    print("Accuracy of the neural network on CIFAR_100 is: %.2f %%" %((correct/total)*100))
    f.write("Accuracy: " + str(((correct/total)*100)) + '\n')
    f.write(str(classes) + '\n')
    f.write(str(confMatrix) + '\n')
    prec, recall = specificMetrics(confMatrix)
    f.write(str(prec) + '\n')
    f.write(str(recall) + '\n')
    f.close()
    return (accLoss/(total/trainLoader.batch_size)), (correct/total)

def specificMetrics(confMatrix):
    """
    Calculates precision and recall from a given confusion
    matrix and returns calculated metrics.
    Parameters
    ----------
    confMatrix: n x n numpy array
        Made from the predictions and true labels of a
        given set of data
    Return
    ------
    precc
        Precision on all classes
    recal 
        Recall on all classes
    """
    precc = np.zeros(np.size(confMatrix, 0))
    recal = np.zeros(np.size(confMatrix, 0))
    for i in range(np.size(confMatrix, 0)):
        tp = 0
        fp = 0
        fn = 0
        for j in range(np.size(confMatrix, 0)):
            if i == j:
                tp += confMatrix[i, j]
            else:
                fn += confMatrix[j, i]
                fp += confMatrix[i, j]
            
        precc[i] += tp/(tp + fp)
        recal[i] += tp/(tp + fn)

    return precc, recal

In [9]:
def plot_training_progress(save_dir, data):
  fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,8))

  linewidth = 2
  legend_size = 10
  train_color = 'm'
  val_color = 'c'

  num_points = len(data['train_loss'])
  x_data = np.linspace(1, num_points, num_points)
  ax1.set_title('Cross-entropy loss')
  ax1.plot(x_data, data['train_loss'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax1.plot(x_data, data['valid_loss'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax1.legend(loc='upper right', fontsize=legend_size)
  ax2.set_title('Average class accuracy')
  ax2.plot(x_data, data['train_acc'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax2.plot(x_data, data['valid_acc'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax2.legend(loc='upper left', fontsize=legend_size)
  ax3.set_title('Learning rate')
  ax3.plot(x_data, data['lr'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='learning_rate')
  ax3.legend(loc='upper left', fontsize=legend_size)

  save_path = os.path.join(save_dir, 'plots/training_plot4.png')
  print('Plotting in: ', save_path)
  plt.savefig(save_path)

In [14]:
def trainNetwork():
    """Performs a standard procedure for training a neural network.
    Training progress after each learning epoch is evaluated in order to
    gain insigth into ConvNets continuous performance.
    Important notes
    ---------------
    Loss function: Cross entropy loss

    Optimizer: Adam
    
    Scheduler: ExponentialLR
    """
    plot_data = {}
    plot_data['train_loss'] = []
    plot_data['valid_loss'] = []
    plot_data['train_acc'] = []
    plot_data['valid_acc'] = []
    plot_data['lr'] = []
    SAVE_DIR = '/content/drive/MyDrive/Zavrsni'
    device = torch.device('cuda')

    net = ResidualNetwork().to(device=device)
    summary(net, (3, 32, 32))

    lossFunc = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    
    for e in range(epoch):
    
        accLoss = 0.0

        for i, data in enumerate(trainLoader, 0):
            inputs, labels = data
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)

            optimizer.zero_grad()

            outputs = net.forward(inputs)
            loss = lossFunc(outputs, labels)
            loss.backward()
            optimizer.step()

            accLoss += loss.item()

            if i % 100 == 0:
                print("Epoch: %d, Iteration: %5d, Loss: %.3f" % ((e + 1), (i), (accLoss / (i + 1))))
                
        train_loss, train_acc = evaluate(net, False, os.path.join(SAVE_DIR, "eval/train_eval4"))
        val_loss, val_acc = evaluate(net, True, os.path.join(SAVE_DIR, "eval/test_eval4"))

        plot_data['train_loss'] += [train_loss]
        plot_data['valid_loss'] += [val_loss]
        plot_data['train_acc'] += [train_acc]
        plot_data['valid_acc'] += [val_acc]
        plot_data['lr'] += [scheduler.get_last_lr()]

        scheduler.step()
    
    val_loss, val_acc = evaluate(net, True, os.path.join(SAVE_DIR, "eval/final_eval4"))
    plot_training_progress(SAVE_DIR, plot_data)
    PATH = os.path.join(SAVE_DIR, "CIFAR_100/cifar_net3.pth")
    torch.save(net.state_dict(), PATH)

#trainNetwork()
device = torch.device('cuda')
net = ResidualNetwork().to(device=device)
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,320
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,320
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
    ResidualBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,320
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,